In [12]:
# GPU details
!nvidia-smi

In [13]:
# CPU details
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.130
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	:

In [14]:
# code won't work without this
!pip install keras --upgrade

Requirement already up-to-date: keras in /opt/conda/lib/python3.6/site-packages (2.3.1)


In [15]:
# suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [16]:
import os # for basic operations like folder creation, directory validation etc
import glob # finds file according to wildcard given
import shutil # for moving files
import matplotlib.pyplot as plt # for plotting graphs and viewing images

In [17]:
from keras.models import load_model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 96, 96
nb_evaluate_samples = 1500
batch_size = 16
epochs = 100
evaluate_data_dir = "/kaggle/input/fingerprint-database/fvc2006SETA/fvc2006SETA"

# functions to calculate f1score
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

custom_objects = {
    'f1score': f1score,
    'precision_m': precision_m,
    'recall_m': recall_m
}

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    validation_split=0.40,
)

evaluation_generator = train_datagen.flow_from_directory(
        evaluate_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        )

model = load_model('/kaggle/input/fingerprint-database//highest_val_accuracy.h5', custom_objects=custom_objects)
model.summary()

history = model.evaluate_generator(evaluation_generator, verbose=1, use_multiprocessing=True)

Found 1680 images belonging to 140 classes.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 47, 47, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 47, 47, 32)   0           batch_normalization_1[0][0]      
________________________________________________

105/105 [==============================] - 19s 178ms/step


In [20]:
print(model.metrics_names)
print(history)
print("Accuracy: " + str(history[1]*100))

['loss', 'accuracy', 'f1score', 'recall_m', 'precision_m']
[0.029652509838342667, 0.987500011920929, 0.9880760312080383, 0.987500011920929, 0.9886904954910278]


TypeError: must be str, not numpy.float64